In [14]:
# Import Library

# For Data Analysis
import os
import numpy as np                 
import pandas as pd      
import numpy as np
import re

# For Google Translate API
from google.cloud import translate_v2 as translate

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/shlf6/P-Stage/durable-bit-366706-a3b98c10e0df.json"

In [15]:
# file path save
FILE_PATH = './data'
train_path = os.path.join(FILE_PATH, 'train.csv')
valid_path = os.path.join(FILE_PATH, 'dev.csv')
test_path = os.path.join(FILE_PATH, 'test.csv')
ss_path = os.path.join(FILE_PATH, 'sample_submission.csv')

# Load dataset
train_df = pd.read_csv(train_path, engine='python')
valid_df = pd.read_csv(valid_path, engine='python')
test_df = pd.read_csv(test_path, engine='python')
ss_df = pd.read_csv(ss_path, engine='python')

RANDOM_SEED = 12
np.random.seed(RANDOM_SEED)
train_df

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0


In [16]:
train_df.iloc[0]['sentence_1']

'스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~'

In [17]:
ko_src = train_df.iloc[0]['sentence_1']

client = translate.Client()
result = client.translate(ko_src, target_language='en')
en_src = result['translatedText']

en_src

'There is a thrill and a twist, and it is different from other Korean movie trash.'

In [18]:
result = client.translate(en_src, target_language='ko')
ko_dst = result['translatedText']

ko_dst

'스릴과 반전이 있고, 다른 한국 영화 쓰레기들과 다르다.'

In [19]:
bins = [-0.1, 0.0, 0.9, 1.9, 2.9, 3.9, 4.9, 5]
df_label_bins = pd.cut(train_df['label'], bins, labels=['0.0','0.1~0.9', '1.0~1.9', '2.0~2.9', 
                                                        '3.0~3.9', '4.0~4.9', '5.0'])

# train_df 복사
new_train_df = train_df.copy()
new_train_df['label-binning'] = df_label_bins
new_train_df['label-binning']

0       2.0~2.9
1       4.0~4.9
2       2.0~2.9
3       3.0~3.9
4           0.0
         ...   
9319    0.1~0.9
9320    4.0~4.9
9321    0.1~0.9
9322    3.0~3.9
9323    1.0~1.9
Name: label-binning, Length: 9324, dtype: category
Categories (7, object): ['0.0' < '0.1~0.9' < '1.0~1.9' < '2.0~2.9' < '3.0~3.9' < '4.0~4.9' < '5.0']

In [20]:
new_train_df['label-binning'].value_counts().sort_index()

0.0        2119
0.1~0.9    1592
1.0~1.9    1368
2.0~2.9    1137
3.0~3.9    1715
4.0~4.9    1302
5.0          91
Name: label-binning, dtype: int64

In [25]:
def back_translation(df, binary_label, cnt) :
    src_df = df[(df['binary-label'] == binary_label)]
    
    index_list = []

    # random_swap은 2개씩 나오기 때문에 cnt // 2
    while len(index_list) < cnt :
        random_index = np.random.randint(len(src_df))

        if random_index not in index_list :
            index_list.append(random_index)
            
    tmp_df = src_df.iloc[index_list].reset_index()
    
    translate_sentence_1 = []
    translate_sentence_2 = []
    
    for i in range(cnt) :
        ko_src1 = tmp_df.iloc[i]['sentence_1']

        client = translate.Client()
        result = client.translate(ko_src1, target_language='en')
        en_src1 = result['translatedText']

        result = client.translate(en_src1, target_language='ko')
        ko_dst1 = result['translatedText']

        ko_src2 = tmp_df.iloc[i]['sentence_2']

        client = translate.Client()
        result = client.translate(ko_src2, target_language='en')
        en_src2 = result['translatedText']

        result = client.translate(en_src2, target_language='ko')
        ko_dst2 = result['translatedText']

        translate_sentence_1.append(ko_dst1)
        translate_sentence_2.append(ko_dst2)
    
    translate_df1 = pd.DataFrame(translate_sentence_1, columns=['translate_sentence_1'])
    translate_df2 = pd.DataFrame(translate_sentence_2, columns=['translate_sentence_2'])
    
    
    return pd.concat([tmp_df, translate_df1, translate_df2], axis=1)

    

In [26]:
%%time
back_translation_df = back_translation(train_df, 0.0, 10)
back_translation_df

CPU times: total: 2.89 s
Wall time: 15.7 s


,index,id,source,sentence_1,sentence_2,label,binary-label,translate_sentence_1,translate_sentence_2
0,692,boostcamp-sts-v1-train-692,nsmc-sampled,왜 이제야 봤을까...,왜 봤을까 후회되는 영화...,1.4,0.0,왜 이제서야 봤지...,보고 후회하는 영화...
1,754,boostcamp-sts-v1-train-754,nsmc-sampled,중간에 끌까말까를 고민하게 한 영화...,뜬금없는 곳에서도 울게해준 영화...,0.2,0.0,중간에 멈출까 말까 고민하게 만든 영화...,의외의 장소에서도 울게 만든 영화...
2,2740,boostcamp-sts-v1-train-2740,nsmc-sampled,가...강호동을 본것같다,만화가 고우영 본좌님의...ㅜㅜ,0.0,0.0,강호동 본거같은데,만화가 고우영의 본편... ㅜㅜ
3,8511,boostcamp-sts-v1-train-8511,nsmc-sampled,후기보고갔다가 증말 ㅠㅠ,애니판보고 봤는데 진짜 슬프다ㅠㅠㅠ,0.0,0.0,후기보고 간증했어요 ㅠㅠ,애니판 봤는데 진짜 슬프다 ㅠㅠ
4,5623,boostcamp-sts-v1-train-5623,petition-sampled,국민연금도 약관대출해주세요.,장애인도 보험가입할 수 있게해주세요.,0.0,0.0,국민연금에서도 빌려주세요.,장애인이 보험에 가입할 수 있도록 해주세요.
5,1869,boostcamp-sts-v1-train-1869,slack-sampled,헉 바로 다리 풀기…,헉… 다음에 저도….,0.0,0.0,헐 다리좀 풀어줘...,으... 다음에는 내가... .
6,5370,boostcamp-sts-v1-train-5370,nsmc-sampled,스토리 이해를 못하면 자기를 탓해야지...,그러나 소재나 스토리가 영...,1.2,0.0,이야기를 이해하지 못하면 스스로를 비난해야합니다 ...,하지만 소재나 스토리는 어리고...
7,6832,boostcamp-sts-v1-train-6832,slack-rtt,아 저도 매일 사용한다로 클릭 ㅋㅋ,오 맨날쓰는데 꾹꾹눌러요 ㅋㅋ,1.0,0.0,오 맨날쓰는데 꾹꾹눌러요 ㅋㅋ,아 맨날 사용하니까 꾹꾹 눌러담아요.
8,162,boostcamp-sts-v1-train-162,slack-sampled,대박! 허리가 훨씬 편안하게 느껴지네요.,하늘색깔이 정말 파랗고 멋지네요.,0.0,0.0,세상에! 허리가 훨씬 편해졌어요.,하늘색이 정말 파랗고 예쁩니다.
9,2181,boostcamp-sts-v1-train-2181,petition-sampled,산업연수생제도의 문제점,고용 촉진 지원금의 문제점,0.2,0.0,산업연수생 제도의 문제점,고용촉진 보조금 문제


In [10]:
# 띄어쓰기 없애는 함수
def remove_space(sentence) :
    new_sentence = sentence.replace(' ', '')
    return new_sentence

# 마지막에 특수문자 추가
def add_punctuations(sentence) :
    punctuations=[".", "!", "?", '..', '!!', '??', '?!']
    random_index = np.random.randint(len(punctuations))
    new_sentence = sentence + punctuations[random_index]
    return new_sentence

# 특수문자 제거
def remove_punctuations(inputString):
    text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inputString)
    text_rmv = text_rmv.strip() # 공백 제거
    return text_rmv

In [27]:
sentence_1_list = back_translation_df['translate_sentence_1'].tolist()
sentence_2_list = back_translation_df['translate_sentence_2'].tolist()

sentence_1_list, sentence_2_list

(['왜 이제서야 봤지...',
  '중간에 멈출까 말까 고민하게 만든 영화...',
  '강호동 본거같은데',
  '후기보고 간증했어요 ㅠㅠ',
  '국민연금에서도 빌려주세요.',
  '헐 다리좀 풀어줘...',
  '이야기를 이해하지 못하면 스스로를 비난해야합니다 ...',
  '오 맨날쓰는데 꾹꾹눌러요 ㅋㅋ',
  '세상에! 허리가 훨씬 편해졌어요.',
  '산업연수생 제도의 문제점'],
 ['보고 후회하는 영화...',
  '의외의 장소에서도 울게 만든 영화...',
  '만화가 고우영의 본편... ㅜㅜ',
  '애니판 봤는데 진짜 슬프다 ㅠㅠ',
  '장애인이 보험에 가입할 수 있도록 해주세요.',
  '으... 다음에는 내가... .',
  '하지만 소재나 스토리는 어리고...',
  '아 맨날 사용하니까 꾹꾹 눌러담아요.',
  '하늘색이 정말 파랗고 예쁩니다.',
  '고용촉진 보조금 문제'])

In [12]:
src_sentence_1 = []
aug_sentence_1 = []


for idx, sentence in enumerate(sentence_1_list) :
    if idx % 2 == 0 :
        tmp = remove_space(sentence)
    else :
        if sentence[-1] in [".", "!", "?"] :
            tmp = remove_punctuations(sentence)
        else :
            tmp = add_punctuations(sentence)
    
    src_sentence_1.append(sentence)
    aug_sentence_1.append(tmp)
    
print(src_sentence_1)
print(aug_sentence_1)

['태극기 혁명을 시작합시다.', '오 좋아 그 벽에 꼭 참여하고 싶다!!!!!', '하늘에서 편히 쉬세요.', '짧은 시간에 재미있고 완성도 높은 드라마입니다.', '인생 역전 복권!!!', '최근에 약혼도 하셨다고 하시던데, 다시 한 번 축하드립니다.', '내가 늘 사랑했던 별들이 나에게 계속 말을 걸면 내 심장은 쿵쾅거린다.', '이곳에서 왕갈비를 먹고 이 장면을 보는 것은 정말 좋다.', '난민으로 인한 승인율 하락', '구세주 속편을 내놓은 이유는?']
['태극기혁명을시작합시다.', '오 좋아 그 벽에 꼭 참여하고 싶다', '하늘에서편히쉬세요.', '짧은 시간에 재미있고 완성도 높은 드라마입니다', '인생역전복권!!!', '최근에 약혼도 하셨다고 하시던데  다시 한 번 축하드립니다', '내가늘사랑했던별들이나에게계속말을걸면내심장은쿵쾅거린다.', '이곳에서 왕갈비를 먹고 이 장면을 보는 것은 정말 좋다', '난민으로인한승인율하락', '구세주 속편을 내놓은 이유는']


In [13]:
_id = translation_df["id"]
source = translation_df["source"]
sentence_1 = pd.Series(src_sentence_1)
sentence_2 = pd.Series(aug_sentence_1)
label = pd.Series([5.0] * len(translation_df))
binarylabel = pd.Series([1.0] * len(translation_df))


aug1_df = pd.DataFrame(
        {
        "id": _id,
        "source": source,
        "sentence_1": sentence_1,
        "sentence_2": sentence_2,
        "label": label,
        "binary-label": binarylabel,
        }        
)

aug_df1

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-2338,petition-sampled,태극기 혁명을 시작합시다.,태극기혁명을시작합시다.,5.0,1.0
1,boostcamp-sts-v1-train-5442,slack-sampled,오 좋아 그 벽에 꼭 참여하고 싶다!!!!!,오 좋아 그 벽에 꼭 참여하고 싶다,5.0,1.0
2,boostcamp-sts-v1-train-6055,nsmc-sampled,하늘에서 편히 쉬세요.,하늘에서편히쉬세요.,5.0,1.0
3,boostcamp-sts-v1-train-2135,nsmc-sampled,짧은 시간에 재미있고 완성도 높은 드라마입니다.,짧은 시간에 재미있고 완성도 높은 드라마입니다,5.0,1.0
4,boostcamp-sts-v1-train-6385,petition-sampled,인생 역전 복권!!!,인생역전복권!!!,5.0,1.0
5,boostcamp-sts-v1-train-7418,slack-sampled,"최근에 약혼도 하셨다고 하시던데, 다시 한 번 축하드립니다.",최근에 약혼도 하셨다고 하시던데 다시 한 번 축하드립니다,5.0,1.0
6,boostcamp-sts-v1-train-458,slack-sampled,내가 늘 사랑했던 별들이 나에게 계속 말을 걸면 내 심장은 쿵쾅거린다.,내가늘사랑했던별들이나에게계속말을걸면내심장은쿵쾅거린다.,5.0,1.0
7,boostcamp-sts-v1-train-089,slack-sampled,이곳에서 왕갈비를 먹고 이 장면을 보는 것은 정말 좋다.,이곳에서 왕갈비를 먹고 이 장면을 보는 것은 정말 좋다,5.0,1.0
8,boostcamp-sts-v1-train-6073,petition-sampled,난민으로 인한 승인율 하락,난민으로인한승인율하락,5.0,1.0
9,boostcamp-sts-v1-train-7489,nsmc-sampled,구세주 속편을 내놓은 이유는?,구세주 속편을 내놓은 이유는,5.0,1.0


In [73]:
def back_translation_augment(back_translation_df, label_value) :
    sentence_1_list = back_translation_df['translate_sentence_1'].tolist()
    sentence_2_list = back_translation_df['translate_sentence_2'].tolist()

    src_sentence_1, src_sentence_2 = [], []
    aug_sentence_1, aug_sentence_2 = [], []


    for idx, sentence in enumerate(sentence_1_list) :
        if idx % 2 == 0 :
            tmp = remove_space(sentence)
        else :
            if sentence[-1] in [".", "!", "?"] :
                tmp = remove_punctuations(sentence)
            else :
                tmp = add_punctuations(sentence)

        src_sentence_1.append(sentence)
        aug_sentence_1.append(tmp)
        
    for idx, sentence in enumerate(sentence_2_list) :
        if idx % 2 == 0 :
            tmp = remove_space(sentence)
        else :
            if sentence[-1] in [".", "!", "?"] :
                tmp = remove_punctuations(sentence)
            else :
                tmp = add_punctuations(sentence)

        src_sentence_2.append(sentence)
        aug_sentence_2.append(tmp)

    _id = back_translation_df["id"]
    source = back_translation_df["source"]
    
    src_sentence_1 = pd.Series(src_sentence_1)
    aug_sentence_1 = pd.Series(aug_sentence_1)
    src_sentence_2 = pd.Series(src_sentence_2)
    aug_sentence_2 = pd.Series(aug_sentence_2)
    
    label = pd.Series([label_value] * len(back_translation_df))
    binarylabel = pd.Series([1.0] * len(back_translation_df))


    aug1_df = pd.DataFrame(
            {
            "id": _id,
            "source": source,
            "sentence_1": src_sentence_1,
            "sentence_2": aug_sentence_1,
            "label": label,
            "binary-label": binarylabel,
            }        
    )

    aug2_df = pd.DataFrame(
            {
            "id": _id,
            "source": source,
            "sentence_1": src_sentence_2,
            "sentence_2": aug_sentence_2,
            "label": label,
            "binary-label": binarylabel,
            }        
    )
    
    return pd.concat([aug1_df, aug2_df])

In [74]:
back_translation_df = back_translation(train_df, 0.0, 1)
back_translation_df

,index,id,source,sentence_1,sentence_2,label,binary-label,translate_sentence_1,translate_sentence_2
0,1648,boostcamp-sts-v1-train-1648,petition-sampled,대우조선해양 지원을 중단하라,대우조선해양 대표이사 인사에 신중을 기해주십시요.,0.8,0.0,대우조선해양 지원 중단,대우조선해양 대표이사 선임에 유의하시기 바랍니다.


In [87]:
from tqdm import tqdm

augment_df = pd.DataFrame()

label_list = [5.0, 4.8, 4.6, 4.5]
cnt_list = [150, 100, 100, 100]
# cnt_list = [2, 2, 2, 2]

for label_value, cnt in tqdm(zip(label_list, cnt_list)) :
    back_translation_df = back_translation(train_df, 0.0, cnt)
    tmp_df = back_translation_augment(back_translation_df, label_value)

    augment_df = pd.concat([augment_df, tmp_df])
    
augment_df

4it [09:52, 148.20s/it]


,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-7260,petition-rtt,나는 일하기 위해 선출되었지만 대통령과 국회의원들은 인민생활법을 통과시켰다.,나는일하기위해선출되었지만대통령과국회의원들은인민생활법을통과시켰다.,5.0,1.0
1,boostcamp-sts-v1-train-623,petition-sampled,항의도 못해?,항의도 못해,5.0,1.0
2,boostcamp-sts-v1-train-1788,petition-sampled,내국인에 대한 외국인 우대정책 폐지,내국인에대한외국인우대정책폐지,5.0,1.0
3,boostcamp-sts-v1-train-022,slack-sampled,속상할 때 보면 속이 터질 것 같아,속상할 때 보면 속이 터질 것 같아!,5.0,1.0
4,boostcamp-sts-v1-train-4322,nsmc-sampled,내 사랑 장난감 이야기,내사랑장난감이야기,5.0,1.0
...,...,...,...,...,...,...
95,boostcamp-sts-v1-train-5160,slack-sampled,한국에서 만나요!!,한국에서 만나요,4.5,1.0
96,boostcamp-sts-v1-train-497,slack-sampled,!!! 당신은 몸이 좋습니까?,!!!당신은몸이좋습니까?,4.5,1.0
97,boostcamp-sts-v1-train-3999,nsmc-sampled,와.. 로그인한지 좀 됐네요.,와 로그인한지 좀 됐네요,4.5,1.0
98,boostcamp-sts-v1-train-8774,nsmc-sampled,그나저나..강풀 원작의 내용을 영화로 번역하기 힘든가요?,그나저나..강풀원작의내용을영화로번역하기힘든가요?,4.5,1.0


In [88]:
new_train_df = pd.concat([train_df, augment_df])
new_train_df

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
...,...,...,...,...,...,...
95,boostcamp-sts-v1-train-5160,slack-sampled,한국에서 만나요!!,한국에서 만나요,4.5,1.0
96,boostcamp-sts-v1-train-497,slack-sampled,!!! 당신은 몸이 좋습니까?,!!!당신은몸이좋습니까?,4.5,1.0
97,boostcamp-sts-v1-train-3999,nsmc-sampled,와.. 로그인한지 좀 됐네요.,와 로그인한지 좀 됐네요,4.5,1.0
98,boostcamp-sts-v1-train-8774,nsmc-sampled,그나저나..강풀 원작의 내용을 영화로 번역하기 힘든가요?,그나저나..강풀원작의내용을영화로번역하기힘든가요?,4.5,1.0


In [89]:
bins = [-0.1, 0.0, 0.9, 1.9, 2.9, 3.9, 4.9, 5]
df_label_bins = pd.cut(new_train_df['label'], bins, labels=['0.0','0.1~0.9', '1.0~1.9', '2.0~2.9', 
                                                        '3.0~3.9', '4.0~4.9', '5.0'])

# train_df 복사
new2_train_df = new_train_df.copy()
new2_train_df['label-binning'] = df_label_bins
new2_train_df

,id,source,sentence_1,sentence_2,label,binary-label,label-binning
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,2.0~2.9
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,4.0~4.9
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,2.0~2.9
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,3.0~3.9
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,0.0
...,...,...,...,...,...,...,...
95,boostcamp-sts-v1-train-5160,slack-sampled,한국에서 만나요!!,한국에서 만나요,4.5,1.0,4.0~4.9
96,boostcamp-sts-v1-train-497,slack-sampled,!!! 당신은 몸이 좋습니까?,!!!당신은몸이좋습니까?,4.5,1.0,4.0~4.9
97,boostcamp-sts-v1-train-3999,nsmc-sampled,와.. 로그인한지 좀 됐네요.,와 로그인한지 좀 됐네요,4.5,1.0,4.0~4.9
98,boostcamp-sts-v1-train-8774,nsmc-sampled,그나저나..강풀 원작의 내용을 영화로 번역하기 힘든가요?,그나저나..강풀원작의내용을영화로번역하기힘든가요?,4.5,1.0,4.0~4.9


In [90]:
new2_train_df['label-binning'].value_counts().sort_index()

0.0        2119
0.1~0.9    1592
1.0~1.9    1368
2.0~2.9    1137
3.0~3.9    1715
4.0~4.9    1902
5.0         391
Name: label-binning, dtype: int64

In [91]:
new2_train_df.to_csv('train_back_translation.csv')  